<a href="https://colab.research.google.com/github/amun83/Financial-Data-Science/blob/master/03-01-Mid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#키워드 뉴스 검색으로 본 감성분석과 관련 종목주가 비교


##구글 드라이브 마운트


*   구글 드라이브에 크롤링한 뉴스 데이터DB를 저장 





In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


##필요 모듈 설치

In [0]:
!pip install -q newspaper3k
!pip install -q finance-datareader
!pip install -q textblob
!pip install -q nltk

    100% |████████████████████████████████| 215kB 6.8MB/s 
    100% |████████████████████████████████| 7.4MB 4.3MB/s 
    100% |████████████████████████████████| 194kB 26.8MB/s 
    100% |████████████████████████████████| 51kB 18.7MB/s 


## 특정 키워드/검색기간으로 금융관련 뉴스 기사 리스트 가져오기


*  네이버 금융 뉴스를 특정 키워드/기간으로 검색한 결과 리스트크롤링(finance.naver.com)
* 키워드:남북경협, 기간:2019.1.1~2019.04.18


In [0]:
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import requests
from urllib import parse
import pandas as pd

def news_article_list(query, start=None, end=None):
  search_url_tmpl = 'https://finance.naver.com/news/news_search.nhn?q=%s&sm=all.basic&pd=4&stDateStart=%s&stDateEnd=%s'
  url = search_url_tmpl % (parse.quote(query.encode('euc-kr')), start, end)
  

  # 마지막 페이지 얻기
  last_page_num = 0 
  
  r = requests.get(url)
  soup = BeautifulSoup(r.text, 'html5lib')
  td_pgrr = soup.find('td', attrs={'class':'pgRR'})
  if td_pgrr: 
      last_page_num = int(td_pgrr.a['href'].split('page=')[1])
      
  row_list = []
  ix = 0
  
  for page in range(1, last_page_num+1):
      paging_url_tmpl = 'https://finance.naver.com/news/news_search.nhn?q=%s&sm=all.basic&pd=4&stDateStart=%s&stDateEnd=%s&page=%s'
      url = paging_url_tmpl % (parse.quote(query.encode('euc-kr')), start, end, page)
  
      r = requests.get(url)
      soup = BeautifulSoup(r.text, "html5lib")
      subjects = soup.find_all('', attrs={'class':'articleSubject'})
      summaries = soup.find_all('', attrs={'class':'articleSummary'})
      for subject, summary in zip(subjects, summaries):
        링크 = 'https://finance.naver.com/' + subject.a['href']
        제목 = subject.a.text.strip()
        제공 = summary.find('span', attrs={'class':'press'}).text.strip()
        날짜 = summary.find('span', attrs={'class':'wdate'}).text.strip()
        검색어 = query
        ix = ix +1
        #print(링크, 제목, 제공, 날짜)
        row_list.append([날짜, 검색어, 제목, 제공, 링크])
          
  df = pd.DataFrame(row_list, columns=['날짜','검색어','제목','제공','링크'])
  df['날짜'] = pd.to_datetime(df['날짜'])
  df.reset_index(inplace=True, drop=True)
  
  return df

articles = news_article_list('남북경협', '2019-01-01', '2019-04-18')


In [0]:
import datetime
articles

,날짜,검색어,제목,제공,링크
0,2019-04-17 16:57:00,남북경협,[장외주식] 싸이버로지텍의 상승세와 폴루스의 눈에 띄는 반등,머니투데이,https://finance.naver.com//news/news_read.nhn?...
1,2019-04-17 09:37:00,남북경협,"경협모멘텀 사라진 건설주, ‘생활SOC’로 탄력받나",헤럴드경제,https://finance.naver.com//news/news_read.nhn?...
2,2019-04-16 19:17:00,남북경협,[중기중앙회 108곳 설문조사] 개성공단 입주기업 86% “공단 폐쇄 전보다 경영...,부산일보,https://finance.naver.com//news/news_read.nhn?...
3,2019-04-16 17:52:00,남북경협,[단독] [톡톡! 부동산] 중진 국회의원도 기획부동산에 당했다,매일경제,https://finance.naver.com//news/news_read.nhn?...
4,2019-04-16 17:24:00,남북경협,"개성공단 기업 98% ""재입주 의향""",파이낸셜뉴스,https://finance.naver.com//news/news_read.nhn?...
5,2019-04-16 15:01:00,남북경협,개성공단 입주기업의 86%가 폐쇄 이후 경영 악화,문화일보,https://finance.naver.com//news/news_read.nhn?...
6,2019-04-16 14:57:00,남북경협,개성공단 기업 `폐쇄 후 경영 악화` 86.2%,매일경제,https://finance.naver.com//news/news_read.nhn?...
7,2019-04-16 14:18:00,남북경협,"'동분서주' 홍남기, '느긋' 아소 다로…한일 경제사령탑 극과극 행보",아시아경제,https://finance.naver.com//news/news_read.nhn?...
8,2019-04-16 11:55:00,남북경협,"생활SOC, 30조원 투입...수혜주를 찾아라",머니투데이,https://finance.naver.com//news/news_read.nhn?...
9,2019-04-16 11:51:00,남북경협,개성공단 입주기업 86.2% “공단 폐쇄 후 경영악화”,조선비즈,https://finance.naver.com//news/news_read.nhn?...


##수집한 뉴스 링크를 모두 다운로드


*   기간이 길어 한꺼번에 다운로드 시 에러가 가끔 에러발생. 



In [0]:
import newspaper

def news_article_single(url):
  article = newspaper.Article(url, language='ko')
  article.download()
  article.parse()
  return article

def news_article_download(articles):
  articles['내용'] = ''
  for ix, row in articles.iterrows():
    
    url = row['링크']
    title = row['제목']
    content = news_article_single(url).text.replace('\n', '')
    articles.loc[ix, '내용'] = content
  return articles
  

In [0]:
articles=pd.concat([articles_201901,articles_201902,articles_201903,articles_201903_1,articles_201903_2,articles_201903_3,articles_201903_4,articles_201903_5,articles_201904], join='outer',ignore_index=True)

In [0]:
articles

,날짜,검색어,제목,제공,링크,내용
0,2019-01-31 20:23:00,남북경협,"`도정공백` 경남, 주력사업 줄타격 불가피",매일경제,https://finance.naver.com//news/news_read.nhn?...,경남도청 전경. 김경수 경남지사가 법정 구속되기 전날인 지난 29일부터 남부내륙철도...
1,2019-01-31 17:19:00,남북경협,"[GOOD 20 & GREAT 20] 남북경협, 4차 산업혁명에 길이 있다",한국경제TV,https://finance.naver.com//news/news_read.nhn?...,[한국경제TV 고영욱 기자]한반도에 훈풍이 불고 있습니다.평창올림픽 단일팀 구성으로...
2,2019-01-31 16:47:00,남북경협,[장외주식] 압타바이오와 신테카바이오의 상승세 지속,머니투데이,https://finance.naver.com//news/news_read.nhn?...,[머니투데이 피스탁 ]31일 코스피시장(2204.85pt)은 기관과 개인은 매도했으...
3,2019-01-31 16:40:00,남북경협,"작년 실적 선방한 5대 건설사, 올해 장사도 잘할까?",뉴스1,https://finance.naver.com//news/news_read.nhn?...,© News1 최수아 디자이너 © News1 최수아 디자이너(서울=뉴스1) 진희정 ...
4,2019-01-31 09:59:00,남북경협,"[특징주] 용평리조트, 남북경협 수혜 기대감 속 상한가",머니S,https://finance.naver.com//news/news_read.nhn?...,용평리조트가 남북경협 수혜 기대감을 모으며 상한가를 달성했다.31일 오전 9시56분...
5,2019-01-31 09:40:00,남북경협,"용평리조트, 남북 경협주로 급부상…'상한가'",머니투데이,https://finance.naver.com//news/news_read.nhn?...,[머니투데이 송지유 기자] [최대주주 통일교재단 대북사업 경험 많아…'대관령-금강산...
6,2019-01-31 09:31:00,남북경협,"[특징주]용평리조트, 통일교재단 부각에 상한가",뉴시스,https://finance.naver.com//news/news_read.nhn?...,【서울=뉴시스】이진영 기자 = 용평리조트가 31일 장 초반 남북경협으로 수혜를 누릴...
7,2019-01-31 09:21:00,남북경협,"[특징주]용평리조트, 새로운 경협株로 떠오르며 상한가",이데일리,https://finance.naver.com//news/news_read.nhn?...,[이데일리 이슬기 기자] 용평리조트(070960)가 남북경협 수혜주 리스트에 새롭게...
8,2019-01-31 09:17:00,남북경협,"[특징주] 용평리조트, 北 관광사업 기대...통일교재단 지분 부각↑",파이낸셜뉴스,https://finance.naver.com//news/news_read.nhn?...,용평리조트가 강세다. 북한 관광사업에 대한 기대감이 높아지는 가운데 통일교재단 지분...
9,2019-01-31 09:14:00,남북경협,"용평리조트, 장초반 상한가...통일교재단 역할론 떠오르며 남북경협주로 부상",서울경제,https://finance.naver.com//news/news_read.nhn?...,용평리조트(070960)[서울경제 바로가기]Telegram으로 서울경제 뉴스를 실시...


## DB 저장


*   구글 드라이브에 크롤링한 뉴스데이터 저장(드라이브\Colab Notebooks\real_trade.bd)



In [0]:
import sqlite3  

conn = sqlite3.connect('/content/gdrive/My Drive/Colab Notebooks/naver_news.db')

articles.to_sql('keyword_news', conn, if_exists='replace')
df_temp = pd.read_sql("select * from keyword_news", conn)
len(df_temp.index)

1772

## 번역 API를 이용하여 영어로 번역 (Papago API 사용)


*   하루에 100만자 제한





In [0]:
import urllib
import json
import re

def translate(content, src='ko', dst='en') :
  client_id = ' '
  client_secret = ' '
  encText = urllib.parse.quote(content)
  data = 'source=' + src + '&target=' + dst + '&text=' + encText
  url = 'https://naveropenapi.apigw.ntruss.com/nmt/v1/translation'
  request = urllib.request.Request(url)
  request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
  request.add_header("X-NCP-APIGW-API-KEY",client_secret)
  response = None
  try:
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
      response_body = response.read()
      jsonobj = json.loads(response_body.decode('utf-8'))
      return jsonobj['message']['result']['translatedText'].replace('\n', '')
    else:
      return None
  except Exception as e:
    print('error : ', e)
    return None
  
trans_text = translate(df_temp.loc[0, '내용'])
trans_text

'Gyeongnam Provincial Government. A large spreadsheet has been on the line to celebrate the exemption of the government\'s preliminary feasibility study on the southern inland railway project since Tuesday, the day before South Gyeongsang Province Gov. Kim Kyung-soo was arrested in court. [Photo courtesy of Gyeongnam Province] Gyeongnam Provincial Government. A large spreadsheet has been on the line to celebrate the exemption of the government\'s preliminary feasibility study on the southern inland railway project since Tuesday, the day before South Gyeongsang Province Gov. Kim Kyung-soo was arrested in court. As South Gyeongsang Province Governor Kim Kyung-soo is in court, it is expected that various major projects in South Gyeongsang Province will inevitably lose steam. Since Kim took office, he has been shifting to an acting post due to the unprecedented hiatus in the provincial government at a time when he has completed various major projects and is set to Gyeongnam Branch of the 2

In [0]:
def news_article_translate(articles):
  articles['번역된내용'] = ''
  for ix, row in articles.iterrows():
    url = row['링크']
    content = row['내용']
    trans_text = translate(row['제목'])
    if (trans_text is None) : 
      print('error : ', row['링크'], row['제목'])
      continue
    # 정제
    trans_text = re.sub('\[.*?\]', '', trans_text)
    trans_text = re.sub('[a-zA-Z0-9_+.-]*?@.*$', '', trans_text)
    articles.loc[ix, '번역된제목'] = trans_text
  return articles
  


In [0]:
news_article_translate(articles)
news_articles

,검색어,날짜,내용,링크,번역된내용,번역된내용_BLOB,번역된제목,제공,제목
0,남북경협,2019-01-31 20:23:00,경남도청 전경. 김경수 경남지사가 법정 구속되기 전날인 지난 29일부터 남부내륙철도...,https://finance.naver.com//news/news_read.nhn?...,,,South Gyeongsang Province is forced to launch ...,매일경제,"`도정공백` 경남, 주력사업 줄타격 불가피"
1,남북경협,2019-01-31 17:19:00,[한국경제TV 고영욱 기자]한반도에 훈풍이 불고 있습니다.평창올림픽 단일팀 구성으로...,https://finance.naver.com//news/news_read.nhn?...,,,"Inter-Korean economic cooperation, there is a...",한국경제TV,"[GOOD 20 & GREAT 20] 남북경협, 4차 산업혁명에 길이 있다"
2,남북경협,2019-01-31 16:47:00,[머니투데이 피스탁 ]31일 코스피시장(2204.85pt)은 기관과 개인은 매도했으...,https://finance.naver.com//news/news_read.nhn?...,,,Continuing the Rise of Aptabio and Sintekavio,머니투데이,[장외주식] 압타바이오와 신테카바이오의 상승세 지속
3,남북경협,2019-01-31 16:40:00,© News1 최수아 디자이너 © News1 최수아 디자이너(서울=뉴스1) 진희정 ...,https://finance.naver.com//news/news_read.nhn?...,,,"Will the five biggest builders, which performe...",뉴스1,"작년 실적 선방한 5대 건설사, 올해 장사도 잘할까?"
4,남북경협,2019-01-31 09:59:00,용평리조트가 남북경협 수혜 기대감을 모으며 상한가를 달성했다.31일 오전 9시56분...,https://finance.naver.com//news/news_read.nhn?...,,,"Yongpyong Resort, South-North Economic Coopera...",머니S,"[특징주] 용평리조트, 남북경협 수혜 기대감 속 상한가"
5,남북경협,2019-01-31 09:40:00,[머니투데이 송지유 기자] [최대주주 통일교재단 대북사업 경험 많아…'대관령-금강산...,https://finance.naver.com//news/news_read.nhn?...,,,Yongpyong Resort and Inter-Korean Economic Coo...,머니투데이,"용평리조트, 남북 경협주로 급부상…'상한가'"
6,남북경협,2019-01-31 09:31:00,【서울=뉴시스】이진영 기자 = 용평리조트가 31일 장 초반 남북경협으로 수혜를 누릴...,https://finance.naver.com//news/news_read.nhn?...,,,Yongpyong Resort and Tongilgyo Foundation's em...,뉴시스,"[특징주]용평리조트, 통일교재단 부각에 상한가"
7,남북경협,2019-01-31 09:21:00,[이데일리 이슬기 기자] 용평리조트(070960)가 남북경협 수혜주 리스트에 새롭게...,https://finance.naver.com//news/news_read.nhn?...,,,"Yongpyong Resort, emerging as a new economic c...",이데일리,"[특징주]용평리조트, 새로운 경협株로 떠오르며 상한가"
8,남북경협,2019-01-31 09:17:00,용평리조트가 강세다. 북한 관광사업에 대한 기대감이 높아지는 가운데 통일교재단 지분...,https://finance.naver.com//news/news_read.nhn?...,,,"Yongpyong Resort, Expecting Tourism Business.....",파이낸셜뉴스,"[특징주] 용평리조트, 北 관광사업 기대...통일교재단 지분 부각↑"
9,남북경협,2019-01-31 09:14:00,용평리조트(070960)[서울경제 바로가기]Telegram으로 서울경제 뉴스를 실시...,https://finance.naver.com//news/news_read.nhn?...,,,"Yongpyong Resort, the upper limit of Jangchoba...",서울경제,"용평리조트, 장초반 상한가...통일교재단 역할론 떠오르며 남북경협주로 부상"


 ## 번역 API를 이용하여 영어로 번역 (textblob)
 ### 여러번 시도했더니  http error 503 service unavailable 에러 발생함

In [0]:
from textblob import TextBlob

def news_article_translate_textblob(articles):
  articles['번역된내용_BLOB'] = ''
  for ix, row in articles.iterrows():
    url = row['링크']
    title = row['제목']
    blob = TextBlob(row['내용'])
    trans_text = str(blob.translate(to='en'))
    # 정제
    trans_text = re.sub('\[.*?\]', '', trans_text)
    trans_text = re.sub('[a-zA-Z0-9_+.-]*?@.*$', '', trans_text)
    articles.loc[ix, '번역된내용_BLOB'] = trans_text
  return articles



## DB에 저장


*   구글 드라이브에 크롤링한 뉴스데이터 저장(드라이브\Colab Notebooks\real_trade.bd)




In [0]:
import sqlite3

conn = sqlite3.connect('/content/gdrive/My Drive/Colab Notebooks/real_trade.db')

articles.to_sql('keyword_news', conn, if_exists='replace')


# TextBlob을 이용한 감성 분석

In [0]:
from textblob import TextBlob

# papago 번역본
analysis = TextBlob(articles.loc[0, '번역된제목'])
posneg = analysis.sentiment.polarity

if posneg == 0:
    print('NEUTRAL', posneg)
if posneg > 0:
    print('POSITIVE', posneg)
elif posneg < 0:
    print('NEGATIVE', posneg)

NEGATIVE -0.11875000000000002


In [0]:
from textblob import TextBlob

# textblob 번역본
analysis = TextBlob(articles.loc[4, '번역된제목'])
posneg = analysis.sentiment.polarity

if posneg == 0:
    print('NEUTRAL', posneg)
if posneg > 0:
    print('POSITIVE', posneg)
elif posneg < 0:
    print('NEGATIVE', posneg)

POSITIVE 0.1


In [0]:
def news_article_polarity_score_textblob(articles):
  articles['polarity_score_en'] = ''
  for ix, row in articles.iterrows():
    analysis = TextBlob(articles.loc[ix, '번역된제목'])
    posneg = analysis.sentiment.polarity
    articles.loc[ix, 'polarity_score_en'] = posneg
  return articles

In [0]:
news_article_polarity_score_textblob(articles)

#KNLPY, KOSAC사전을 이용한 감성분석

In [0]:
pip install konlpy 

    100% |████████████████████████████████| 19.4MB 1.7MB/s 
    100% |████████████████████████████████| 174kB 30.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/0e/2b/e8/c0b818ac4b3d35104d35e48cdc7afe27fc06ea277feed2831a
Successfully built JPype1


In [0]:
!git clone https://github.com/FinanceData/corpus.git

Cloning into 'corpus'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 11 (delta 1), reused 11 (delta 1), pack-reused 0
Unpacking objects: 100% (11/11), done.


In [0]:
import pandas as pd
from konlpy.tag import Kkma

engine = Kkma()
polarity_dic = pd.read_csv('corpus/kosac/polarity.csv')
polarity_dic.set_index('ngram', inplace=True)

def polarity_score(text):
    pos_tags = engine.pos(text)

    # n-gram
    unigram = ['/'.join(p) for p in pos_tags]
    bigram = [';'.join(z) for z in zip(unigram, unigram[1:])]
    trigram = [';'.join(z) for z in zip(*[unigram[i:] for i in range(3)])]
    
    p_score = 0
    # polarity score = (p-n) / (p+n)
    for ngram in [unigram, bigram, trigram]:
        cond = polarity_dic.index.isin(ngram)
        s = polarity_dic.loc[cond, :].sum()
        if s['POS'] + s['NEG'] > 0: # div 0 방지
            p_score += (s['POS'] - s['NEG']) / (s['POS'] + s['NEG'])
    return p_score

In [0]:
def news_article_polarity_score(articles):
  articles['polarity_score'] = ''
  for ix, row in articles.iterrows():
    content = row['내용']
    pscore=polarity_score(content)
    articles.loc[ix, 'polarity_score'] = pscore
  return articles

In [0]:
news_article_polarity_score(articles)

In [0]:
articles

,검색어,날짜,내용,링크,번역된제목,제공,제목,polarity_score,polarity_score_en,Date
0,남북경협,2019-01-31 20:23:00,경남도청 전경. 김경수 경남지사가 법정 구속되기 전날인 지난 29일부터 남부내륙철도...,https://finance.naver.com//news/news_read.nhn?...,South Gyeongsang Province is forced to launch ...,매일경제,"`도정공백` 경남, 주력사업 줄타격 불가피",-0.74296,-0.11875,
1,남북경협,2019-01-31 17:19:00,[한국경제TV 고영욱 기자]한반도에 훈풍이 불고 있습니다.평창올림픽 단일팀 구성으로...,https://finance.naver.com//news/news_read.nhn?...,"Inter-Korean economic cooperation, there is a...",한국경제TV,"[GOOD 20 & GREAT 20] 남북경협, 4차 산업혁명에 길이 있다",0.489371,0.1,
2,남북경협,2019-01-31 16:47:00,[머니투데이 피스탁 ]31일 코스피시장(2204.85pt)은 기관과 개인은 매도했으...,https://finance.naver.com//news/news_read.nhn?...,Continuing the Rise of Aptabio and Sintekavio,머니투데이,[장외주식] 압타바이오와 신테카바이오의 상승세 지속,0.933616,0,
3,남북경협,2019-01-31 16:40:00,© News1 최수아 디자이너 © News1 최수아 디자이너(서울=뉴스1) 진희정 ...,https://finance.naver.com//news/news_read.nhn?...,"Will the five biggest builders, which performe...",뉴스1,"작년 실적 선방한 5대 건설사, 올해 장사도 잘할까?",1.59582,0,
4,남북경협,2019-01-31 09:59:00,용평리조트가 남북경협 수혜 기대감을 모으며 상한가를 달성했다.31일 오전 9시56분...,https://finance.naver.com//news/news_read.nhn?...,"Yongpyong Resort, South-North Economic Coopera...",머니S,"[특징주] 용평리조트, 남북경협 수혜 기대감 속 상한가",0.362842,0.1,
5,남북경협,2019-01-31 09:40:00,[머니투데이 송지유 기자] [최대주주 통일교재단 대북사업 경험 많아…'대관령-금강산...,https://finance.naver.com//news/news_read.nhn?...,Yongpyong Resort and Inter-Korean Economic Coo...,머니투데이,"용평리조트, 남북 경협주로 급부상…'상한가'",0.558958,0.025,
6,남북경협,2019-01-31 09:31:00,【서울=뉴시스】이진영 기자 = 용평리조트가 31일 장 초반 남북경협으로 수혜를 누릴...,https://finance.naver.com//news/news_read.nhn?...,Yongpyong Resort and Tongilgyo Foundation's em...,뉴시스,"[특징주]용평리조트, 통일교재단 부각에 상한가",0.396934,0,
7,남북경협,2019-01-31 09:21:00,[이데일리 이슬기 기자] 용평리조트(070960)가 남북경협 수혜주 리스트에 새롭게...,https://finance.naver.com//news/news_read.nhn?...,"Yongpyong Resort, emerging as a new economic c...",이데일리,"[특징주]용평리조트, 새로운 경협株로 떠오르며 상한가",1.10548,0.112121,
8,남북경협,2019-01-31 09:17:00,용평리조트가 강세다. 북한 관광사업에 대한 기대감이 높아지는 가운데 통일교재단 지분...,https://finance.naver.com//news/news_read.nhn?...,"Yongpyong Resort, Expecting Tourism Business.....",파이낸셜뉴스,"[특징주] 용평리조트, 北 관광사업 기대...통일교재단 지분 부각↑",0.0756414,0,
9,남북경협,2019-01-31 09:14:00,용평리조트(070960)[서울경제 바로가기]Telegram으로 서울경제 뉴스를 실시...,https://finance.naver.com//news/news_read.nhn?...,"Yongpyong Resort, the upper limit of Jangchoba...",서울경제,"용평리조트, 장초반 상한가...통일교재단 역할론 떠오르며 남북경협주로 부상",0.941262,0.1,


## 날짜별 polarity score 평균값 계산

In [0]:
articles['Date']=pd.to_datetime(articles['날짜'])
articles['Date']=articles['Date'].dt.strftime('%Y%m%d')

In [0]:

articles['polarity_score'] = articles['polarity_score'].astype(float)
articles.dtypes
articles_temp=articles

scores=articles_temp['polarity_score'].groupby(articles_temp['Date']).mean()
scores.columns = ['Date', 'scores']
pscores=pd.DataFrame({'Date':scores.index, 'scores':scores.values})
pscores
pscores['Date'] =  pd.to_datetime(pscores['Date'], format='%Y%m%d')
pscores

,Date,scores
0,2019-01-01,0.785279
1,2019-01-02,0.717636
2,2019-01-03,0.633593
3,2019-01-04,0.532118
4,2019-01-05,1.341311
5,2019-01-06,0.255210
6,2019-01-07,0.845711
7,2019-01-08,0.818305
8,2019-01-09,0.881443
9,2019-01-10,0.929350


##관련 종목과 연관관계 분석
###correlation 계산


*   현대로템
*   현대엘리베이터
*   삼성전자





In [0]:
import FinanceDataReader as fdr
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('figure', figsize=(16, 4))
import datetime
import pandas as pd
start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2019, 4, 18)

# 현대로템(064350)
df = fdr.DataReader("064350", start, end)
df.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2019-01-02,28350,28550,26650,26750,1621370,-0.039497
2019-01-03,27000,29150,26650,28450,2698290,0.063551
2019-01-04,28200,28300,27600,28100,1072464,-0.012302
2019-01-07,28600,28850,28200,28250,1105569,0.005338
2019-01-08,28450,29450,28350,28700,2009963,0.015929


In [0]:

mergepd=pd.merge(pscores,df, on='Date')
mergepd['scores'].corr(mergepd['Close'])

0.169000011483108

In [0]:
# 삼성전자(005930)
df = fdr.DataReader("005930", start, end)
df.tail()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2019-04-12,46050,46900,46000,46850,7668936,0.012973
2019-04-15,47150,47500,47000,47050,8733019,0.004269
2019-04-16,47400,47400,46800,47250,7730881,0.004251
2019-04-17,47300,47600,47000,47050,5513658,-0.004233
2019-04-18,47200,47250,45500,45600,10825043,-0.030818


In [0]:
mergepd=pd.merge(pscores,df, on='Date')
mergepd['scores'].corr(mergepd['Close'])

-0.3398948505403177

In [0]:
# 현대건설(017800)
df = fdr.DataReader("017800", start, end)
df.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2019-01-02,111000,112000,102500,103500,972650,-0.032710
2019-01-03,104500,112500,102000,107500,1421496,0.038647
2019-01-04,106500,110500,105000,110500,661062,0.027907
2019-01-07,111500,113500,110000,110000,597766,-0.004525
2019-01-08,110500,115500,108500,111500,1025885,0.013636


In [0]:
mergepd=pd.merge(pscores,df, on='Date')
mergepd['scores'].corr(mergepd['Close'])

0.10807466519435187

## 결과에 대한 분석
###  결과분석
* correlation r이 +0.1과 +0.3 사이이면, 약한 양적 선형관계(https://en.wikipedia.org/wiki/Correlation_and_dependence)

### 수치가 예상보다 낮은 원인 분석
*   실제 '남북경협'과 연관성이 적은 뉴스 데이터 필터링 부재
*   짧은 뉴스 검색 기간(4개월)

